# Getting started


In [9]:
using Pkg
Pkg.activate("..")
using MolecularGraph

## Retrieve chemical data from PubChem

- PubChem provides REST API access that enables us to search chemicals by their name, chemical properties, structures and other attributes via HTTP.
- `MolecularGraph.pubchemsdf` fetches a SDFile record of a compound from PubChem database by compound ID (PubChem CID) and puts it into temporary cache directory (constant `MolecularGraph.pubchemdir` is set to the path of the default cache store.

In [2]:
pubchemsdf("6437877", "Cefditoren Pivoxil");

Downloading: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/6437877/SDF


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9401    0  9401    0     0   2759      0 --:--:--  0:00:03 --:--:--  2759


- `sdftomol` converts SDFile text into molecular object.
- `drawsvg` generates SVG format image in the given size (width=300, height=300).
- Then, `display` the molecule on the notebook.

In [3]:
pcmol = sdftomol(joinpath(pubchem_dir, "Cefditoren Pivoxil.mol"))
mol_svg = drawsvg(pcmol, 300, 300)
display("image/svg+xml",  mol_svg)

- There are so many trivial hydrogens.
- Hydrogens can be removed by `makehydrogensimplicit`.

In [4]:
mol = graphmol(makehydrogensimplicit(pcmol))
mol_svg = drawsvg(mol, 300, 300)
display("image/svg+xml",  mol_svg)

## Calculate molecular properties

- `molweight` shows standard molecular weight.
- `hacceptorcount` shows number of Hydrogen bond acceptors (F, O or N).
- `hdonorcount` shows number of Hydrogen bond donors (O or N attached to at least one hydrogen).
- `wclogp` shows predicted LogP value by Wildman and Crippen method. 
- `rotatablecount` shows number of rotatable bonds.

In [5]:
println("Molecular weight: ", molweight(mol))
println("Hydrogen acceptors: ", hacceptorcount(mol))
println("Hydrogen donors: ", hdonorcount(mol))
println("LogP: ", wclogp(mol))
println("Rotatable bonds: ", rotatablecount(mol))

Molecular weight: 620.72
Hydrogen acceptors: 13
Hydrogen donors: 2
LogP: 2.3
Rotatable bonds: 13


 ## Show atom indices

- `drawsvg` method is a convenient method that generates `SvgCanvas` object, calls `draw2d!` to set molecule components, and then calls `tosvg` to finalize SVG image.
- `drawatomindex!` adds atom index to the molecular image canvas.
-  '!' of `draw2d!` and `drawatomindex!` is a Julia language convention which means the function is destructive. In this case, `draw2d!` modifies `SvgCanvas` by adding molecule drawing settings and components.
- The default atom indices are the same order as the atom record in SDFile.

In [6]:
canvas = SvgCanvas()
draw2d!(canvas, mol)
drawatomindex!(canvas, mol)
mol_svg = tosvg(canvas, 400, 400)
display("image/svg+xml",  mol_svg)

## Functional group detection

- MolecularGraph.jl provides a functional group detection procedure based on terminology graph.
- It is like a Gene Ontology term graph, but whose nodes are functional group terms and edges are relationship.

In [7]:
fg = functionalgroupgraph(mol)

display("image/svg+xml",  mol_svg)

for (term, components) in fg.componentmap
    nodes = [sort(collect(comp)) for comp in components]
    println("Group: $(string(term)): ", nodes...)
end

Group: Diene: [20, 22, 24, 27]
Group: Oxime: [9, 13, 26]
Group: SecAmine: [12, 18, 25]
Group: SixMemberedRing: [1, 11, 17, 20, 21, 22]
Group: Imine: [13, 26]
Group: Carbonyl: [7, 25][10, 32][6, 23][4, 19]
Group: AromaticSulfur: [2, 30, 40][3, 37, 39]
Group: FiveMemberedRing: [2, 15, 30, 36, 40][3, 14, 31, 37, 39]
Group: Thioether: [1, 17, 21]
Group: FourMemberedRing: [11, 17, 18, 19]
Group: BetaLactam: [4, 11, 17, 18, 19]
Group: TertAmine: [11, 17, 19, 20]
Group: Alkene: [24, 27][20, 22]
Group: Acetal: [5, 8, 23, 28, 32]
Group: Thiazole: [2, 15, 30, 36, 40][3, 14, 31, 37, 39]
Group: Amide: [7, 12, 25][4, 11, 19]
Group: Ester: [8, 10, 28, 32][5, 6, 23, 28]
Group: Azetidine: [11, 17, 18, 19]
Group: Ether: [8, 28, 32][5, 23, 28]
Group: AminoAcid: [4, 12, 18, 19][6, 11, 20, 23]
Group: AromaticNitrogen: [14][15]
Group: AromaticAmine: [16, 39]
Group: CyclicAmine: [11, 19, 20][11, 17, 19][11, 17, 20]


- Some of these terms are subset of others (for example, thiazole "is a" five membered ring). Largest in size and most specified terms often give us important information.
- `largest components`enriches functional group information by collecting largest and specified terms from the functional group graph.

In [8]:
display("image/svg+xml",  mol_svg)
for (term, components) in largestcomponents(fg)
    nodes = [sort(collect(comp)) for comp in components]
    if !isempty(nodes)
        println("Group: $(string(term)): ", nodes...)
    end
end

Group: Oxime: [9, 13, 26]
Group: SixMemberedRing: [1, 11, 17, 20, 21, 22]
Group: SecAmine: [12, 18, 25]
Group: Thioether: [1, 17, 21]
Group: FourMemberedRing: [11, 17, 18, 19]
Group: BetaLactam: [4, 11, 17, 18, 19]
Group: TertAmine: [11, 17, 19, 20]
Group: Acetal: [5, 8, 23, 28, 32]
Group: Thiazole: [2, 15, 30, 36, 40][3, 14, 31, 37, 39]
Group: Amide: [7, 12, 25]
Group: Ester: [8, 10, 28, 32][5, 6, 23, 28]
Group: AminoAcid: [4, 12, 18, 19][6, 11, 20, 23]
Group: CyclicAmine: [11, 19, 20][11, 17, 20]
Group: Diene: [20, 22, 24, 27]
Group: AromaticAmine: [16, 39]
